## Decision Analytics Final Project
### Question: What is the best combination of tourist attractions in NYC that a travel agency can put together in order to provide the best tour experience? 
#### Team Members: Megan, Jhanvi, Hannah, Kaia

In [1]:
import numpy as np
from gurobipy import *
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# prepare model parameters and sets

# parameters
p = [400, 500, 800, 1000] # subject to change when incorporating uncertainty here
pr = np.genfromtxt('Pb1_prob.csv', delimiter=',', encoding='utf-8-sig')  # (1000,)
d = np.genfromtxt('Pb1_D_stochastic.csv', delimiter=',', encoding='utf-8-sig')  # (4, 1000)

# define index and sets
scenario_set = range(len(pr))  # s, range(0, 1000)
class_set = range(len(d[:, 0]))  # c, range(0, 4)

In [ ]:
# initialize model

model = Model()

In [ ]:
# define the decision variables with their constraints

a = model.addVars(class_set, lb=0, vtype=GRB.INTEGER)  # (40, 1) binary variables
x = model.addVars(class_set, scenario_set, lb=0, vtype=GRB.INTEGER)  # (4, 1000) binary variables

In [ ]:
# set up objective function

pb1_objective = sum(pr[s]*sum(p[c]*x[c,s] for c in class_set) for s in scenario_set)
model.setObjective((pb1_objective), GRB.MAXIMIZE)

In [ ]:
# setup remaining constraints.

# The total capacity of the aircraft cannot be gone over the 190 economy seats
model.addConstr(a[0] + 1.2*a[1] + 1.5*a[2] + 2*a[3] <= 190)

# The number of seats sold for each class c must not go over the assigned capacity a for all scenarios s
for c in class_set:
    for s in scenario_set:
        model.addConstr(x[c,s] <= a[c])

# The number of seats sold for each class c must not go over the demand for class c for all scenarios s
for c in class_set:
    for s in scenario_set:
        model.addConstr(x[c,s] <= d[c,s])

In [ ]:
# Solve the problem

model.optimize()

In [ ]:

# Parameters (example data, update based on your specific inputs)
attractions = [
    {"id": 1, "cost": 0, "happiness": 2, "accessibility": 1, "family_friendly": 1},  # Example: Central Park
    {"id": 2, "cost": 33, "happiness": 3, "accessibility": 0, "family_friendly": 1},  # Example: 9/11 Memorial
    # Add all attractions here with relevant attributes
]

budget = 200  # Total budget constraint
max_duration = 8  # Maximum number of hours available for the tour
travel_time = np.random.randint(1, 3, size=(len(attractions), len(attractions)))  # Travel time matrix (example)
demographic_weights = [0.3, 0.4, 0.3]  # Example weights for demographic groups
scenario_probabilities = [0.5, 0.5]  # Probabilities for scenarios (adjust as needed)

# Initialize Model
model = Model("NYC_Tour_Optimization")

# Decision Variables
A = model.addVars(len(attractions), vtype=GRB.BINARY, name="A")  # Binary decision for each attraction

# Objective Function
happiness_scores = [
    demographic_weights[j] * scenario_probabilities[s] * attractions[i]["happiness"]
    for i in range(len(attractions))
    for j in range(len(demographic_weights))
    for s in range(len(scenario_probabilities))
]

model.setObjective(quicksum(A[i] * happiness_scores[i] for i in range(len(attractions))), GRB.MAXIMIZE)

# Constraints

# Budget Constraint
model.addConstr(quicksum(A[i] * attractions[i]["cost"] for i in range(len(attractions))) <= budget, "Budget")

# Total Duration Constraint (tour cannot exceed available time)
attraction_durations = [1.5 for _ in range(len(attractions))]  # Example durations for each attraction in hours
model.addConstr(
    quicksum(A[i] * attraction_durations[i] for i in range(len(attractions))) <= max_duration, "Duration"
)

# Accessibility Constraint (at least 80% of selected attractions must be accessible)
model.addConstr(
    quicksum(A[i] * attractions[i]["accessibility"] for i in range(len(attractions)))
    >= 0.8 * quicksum(A[i] for i in range(len(attractions))),
    "Accessibility"
)

# Family-Friendliness Constraint (at least 50% of selected attractions must be family-friendly)
model.addConstr(
    quicksum(A[i] * attractions[i]["family_friendly"] for i in range(len(attractions)))
    >= 0.5 * quicksum(A[i] for i in range(len(attractions))),
    "Family_Friendly"
)

# Pairwise Constraints (if one attraction is selected, another must also be selected)
model.addConstr(A[0] + A[1] <= 1, "Pairwise_Constraint")  # Example: Cannot select both Central Park and 9/11 Memorial

# Optimize
model.optimize()

# Print Results
if model.status == GRB.OPTIMAL:
    selected_attractions = [i for i in range(len(attractions)) if A[i].X > 0.5]
    print("Optimal Attractions:", selected_attractions)
    print("Maximum Happiness:", model.objVal)
else:
    print("No optimal solution found.")
